In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error
from dmba import stepwise_selection
from dmba import AIC_score
from scipy.stats import kstest
import statsmodels.regression.linear_model as rg
import statsmodels.stats.diagnostic as dg
from statsmodels.stats.stattools import durbin_watson
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white
from sklearn.feature_selection import f_regression

In [96]:
insurance=pd.read_csv(r'C:\Users\hp RED\AIfundamentals\Datos\insurance.csv',delimiter=',')
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [97]:
#Información de mi base de datos
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [98]:
#Revisión de espacios vacíos
insurance.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [99]:
#duplicates = []
#for col in range(insurance.shape[1]):
 #   contents = insurance.iloc[:, col]
    
#    for comp in range(col + 1, insurance.shape[1]):
#        if contents.equals(insurance.iloc[:, comp]):
 #           duplicates.append(comp)
#duplicates = np.unique(duplicates).tolist()
#duplicates

In [100]:
insurance=pd.get_dummies(insurance,columns=['sex'])
insurance

,age,bmi,children,smoker,region,charges,sex_female,sex_male
0,19,27.900,0,yes,southwest,16884.92400,1,0
1,18,33.770,1,no,southeast,1725.55230,0,1
2,28,33.000,3,no,southeast,4449.46200,0,1
3,33,22.705,0,no,northwest,21984.47061,0,1
4,32,28.880,0,no,northwest,3866.85520,0,1
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,no,northwest,10600.54830,0,1
1334,18,31.920,0,no,northeast,2205.98080,1,0
1335,18,36.850,0,no,southeast,1629.83350,1,0
1336,21,25.800,0,no,southwest,2007.94500,1,0


In [101]:
insurance=pd.get_dummies(insurance,columns=['smoker'])
insurance

,age,bmi,children,region,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,southwest,16884.92400,1,0,0,1
1,18,33.770,1,southeast,1725.55230,0,1,1,0
2,28,33.000,3,southeast,4449.46200,0,1,1,0
3,33,22.705,0,northwest,21984.47061,0,1,1,0
4,32,28.880,0,northwest,3866.85520,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,northwest,10600.54830,0,1,1,0
1334,18,31.920,0,northeast,2205.98080,1,0,1,0
1335,18,36.850,0,southeast,1629.83350,1,0,1,0
1336,21,25.800,0,southwest,2007.94500,1,0,1,0


In [102]:
insurance=pd.get_dummies(insurance,columns=['region'])
insurance

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [103]:
data=insurance
data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


## Modelo level-level

In [104]:
predictors=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes','region_northeast','region_northwest','region_southeast','region_southwest']
outcome=['charges']
insurance_lm=LinearRegression()
insurance_lm.fit(data[predictors],data[outcome])
print('Intercept',insurance_lm.intercept_)
coeficientes=insurance_lm.coef_.tolist()[0]
for name, coef in zip(predictors,coeficientes):
    print(f'{name}:{coef}')

Intercept [-666.93771994]
age:256.8563525373485
bmi:339.1934536108376
children:475.50054514913427
sex_female:65.65717969755018
sex_male:-65.6571796975533
smoker_no:-11924.26727095642
smoker_yes:11924.267270956423
region_northeast:587.0092350283243
region_northwest:234.04533560368165
region_southeast:-448.012814359502
region_southwest:-373.04175627250333


In [105]:
model=sm.OLS(data[outcome],data[predictors].assign(const=1))
results=model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:31   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                256.8564     11.899     21.587      0.000     233.514     280.199
bmi                339.1935     28.599     11.860      0.000     283.088     395.298
children           475.5005    137.804      3.451      0.001     205.163     745.838
sex_female         -82.5512    269.226     -0.307      0.759    -610.706     445.604
sex_male          -213.8656    274.976     -0.778      0.437    -753.299     325.568
smoker_no        -1.207e+04    282.338    -42.759      0.000   -1.26e+04   -1.15e+04
smoker_yes        1.178e+04    313.530     37.560      0.000    1.12e+04    1.24e+04
region_northeast   512.9050    300.348      1.708      0.088     -76.303    1102.113
region_northwest   159.9411    301.334      0.531      0.596    -431.201     751.083
region_southeast  -522.1170    330.759     -1.579      0.115   -1170.983     126.749
region_southwest  -447.1459    310.933     -1.438      0.151   -1057.119     162.827
const             -296.4168    430.507     -0.689      0.491   -1140.964     548.130
==============================================================================
Omnibus:                      300.366   Durbin-Watson:                   2.088
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              718.887
Skew:                           1.211   Prob(JB):                    7.86e-157
Kurtosis:                       5.651   Cond. No.                     5.17e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.32e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [106]:
rows=len(data.index)
fitted = insurance_lm.predict(data[predictors])
RMSE = np.sqrt(mean_squared_error(data[outcome], fitted))
r2 = r2_score(data[outcome], fitted)
print(f'RMSE: {RMSE:.0f}')
print(f'r2: {r2:.4f}')
print(rows)
adj_r2=1-(1-r2)*((rows-1)/(rows-len(predictors)-1))
print(f'adj_r2:{adj_r2:.4f}')

RMSE: 6042
r2: 0.7509
1338
adj_r2:0.7488


In [107]:
#stepwise selection


y=data[outcome]
X=data[predictors]

def train_model(variables):
    if len(variables)==0:
        return None
    model=LinearRegression()
    model.fit(X[variables],y)
    return model

def score_model(model,variables):
    if len(variables)==0:
        return AIC_score(y,[y.mean()]*len(y),model,df=1)
    return AIC_score(y,model.predict(X[variables]),model)

best_model,best_variables= stepwise_selection(X.columns, train_model,score_model,verbose=True)
print(best_model.intercept_)
print('Coefficients')
for name, coef in zip(best_variables,best_model.coef_):
    print(name,coef)

Variables: age, bmi, children, sex_female, sex_male, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest
Start: score=28959.26, constant
Step: score=27667.46, add smoker_no
Step: score=27251.32, add age
Step: score=27119.84, add bmi
Step: score=27108.04, add children
Step: score=27104.10, add region_northeast
Step: score=27101.69, add region_northwest
Step: score=27101.53, add sex_female
Step: score=27101.51, add region_southeast
Step: score=27101.51, unchanged None
[10818.63061505]
Coefficients
smoker_no [-23848.53454191    256.85635254    339.19345361    475.50054515
    960.0509913     607.08709188    131.3143594     -74.97105809]


In [108]:
predictors=['smoker_no','age','bmi','children','region_northeast','region_northwest','sex_female','region_southeast']
outcome=['charges']
model2=sm.OLS(data[outcome],data[predictors].assign(const=1))
results2=model2.fit()
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:33   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
smoker_no        -2.385e+04    413.153    -57.723      0.000   -2.47e+04    -2.3e+04
age                256.8564     11.899     21.587      0.000     233.514     280.199
bmi                339.1935     28.599     11.860      0.000     283.088     395.298
children           475.5005    137.804      3.451      0.001     205.163     745.838
region_northeast   960.0510    477.933      2.009      0.045      22.466    1897.636
region_northwest   607.0871    477.204      1.272      0.204    -329.068    1543.242
sex_female         131.3144    332.945      0.394      0.693    -521.842     784.470
region_southeast   -74.9711    470.639     -0.159      0.873    -998.247     848.305
const             1.082e+04   1071.815     10.094      0.000    8715.997    1.29e+04
==============================================================================
Omnibus:                      300.366   Durbin-Watson:                   2.088
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              718.887
Skew:                           1.211   Prob(JB):                    7.86e-157
Kurtosis:                       5.651   Cond. No.                         343.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
predictors1=['smoker_no','age','bmi','children','region_northeast']
outcome1=['charges']
model3=sm.OLS(data[outcome1],data[predictors1].assign(const=1))
results3=model3.fit()
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     801.0
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:34   Log-Likelihood:                -13549.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1332   BIC:                         2.714e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
smoker_no        -2.381e+04    410.773    -57.959      0.000   -2.46e+04    -2.3e+04
age                257.4058     11.885     21.657      0.000     234.090     280.722
bmi                329.4629     27.616     11.930      0.000     275.287     383.639
children           479.5142    137.674      3.483      0.001     209.432     749.596
region_northeast   773.9462    390.709      1.981      0.048       7.473    1540.419
const              1.13e+04   1011.930     11.163      0.000    9310.891    1.33e+04
==============================================================================
Omnibus:                      302.023   Durbin-Watson:                   2.083
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              727.655
Skew:                           1.215   Prob(JB):                    9.81e-159
Kurtosis:                       5.674   Cond. No.                         317.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [110]:
insurance_lm2=LinearRegression()
insurance_lm2.fit(data[predictors1],data[outcome1])
print('Intercept',insurance_lm2.intercept_)
coeficientes=insurance_lm2.coef_.tolist()[0]
for name, coef in zip(predictors,coeficientes):
    print(f'{name}:{coef}')

Intercept [11296.04039768]
smoker_no:-23808.13494441841
age:257.4058325513014
bmi:329.46287027714857
children:479.5141858921514
region_northeast:773.9462167676388


In [111]:
rows1=len(data.index)
fitted1 = insurance_lm2.predict(data[predictors1])
RMSE1 = np.sqrt(mean_squared_error(data[outcome1], fitted))
r21 = r2_score(data[outcome1], fitted)
print(f'RMSE: {RMSE1:.0f}')
print(f'r2: {r21:.4f}')
print(rows1)
adj_r2=1-(1-r21)*((rows1-1)/(rows1-len(predictors1)-1))
print(f'adj_r2:{adj_r2:.4f}')

RMSE: 6042
r2: 0.7509
1338
adj_r2:0.7500


## Cross-Validation

In [112]:
k=10

In [113]:
slices=np.array_split(data,k)
slices

[     age     bmi  children      charges  sex_female  sex_male  smoker_no  \
 0     19  27.900         0  16884.92400           1         0          0   
 1     18  33.770         1   1725.55230           0         1          1   
 2     28  33.000         3   4449.46200           0         1          1   
 3     33  22.705         0  21984.47061           0         1          1   
 4     32  28.880         0   3866.85520           0         1          1   
 ..   ...     ...       ...          ...         ...       ...        ...   
 129   38  34.700         2   6082.40500           0         1          1   
 130   59  26.505         0  12815.44495           1         0          1   
 131   61  22.040         0  13616.35860           1         0          1   
 132   53  35.900         2  11163.56800           1         0          1   
 133   19  25.555         0   1632.56445           0         1          1   
 
      smoker_yes  region_northeast  region_northwest  region_southeast  \


In [114]:
rsquared_results = []
f_statistic_results = []
p_values_results = []

for i in range(len(slices)):
    reserved_data = slices[i]
    left_side = slices[:i]
    right_side = slices[i+1:]
    remaining_data = pd.concat(left_side+right_side) # Reto: Unir dos partes

    #Entrenamiento
    model = LinearRegression()
    model.fit(remaining_data[predictors1],remaining_data[outcome1])

    rsquared = model.score(reserved_data[predictors1],reserved_data[outcome1])
    f_statistic,p_values = f_regression(reserved_data[predictors1],reserved_data[outcome1])

    rsquared_results.append(rsquared)
    f_statistic_results.append(f_statistic)
    p_values_results.append(p_values)

C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

In [115]:
rsquared_results # Promediar los resultados
mean_r2 = sum(rsquared_results)/len(rsquared_results)
mean_r2

0.7451270756583329

In [116]:
#Prueba de linealidad de Ramsey
reg=rg.OLS(data[outcome1],data[predictors1],hasconst=bool).fit()
reset=dg.linear_reset(reg,power=2,test_type='fitted',use_f=True)
print('Ramsay_Reset Test P-Value:', np.round(reset.pvalue,6))
#El p-value es menor a 0.05 entonces existe suficiente evidencia de que la forma funcional NO
#es correcta, esto se puede deber a una mala especificación del modelo o variables omitidas

Ramsay_Reset Test P-Value: 0.0


C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\statsmodels\stats\diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


In [117]:
vif_data=pd.DataFrame()
vif_data["feature"]=data[predictors1].columns
vif_data["VIF"]=[variance_inflation_factor(data[predictors1].values,i)
                for i in range(len(data[predictors1].columns))]
print(vif_data)
#Si el valor es mayor a 10, significa que la varianza de las variables incrementa mucho con el tiempo,
#de esta forma, provocaría multicolinealidad en el modelo

            feature       VIF
0         smoker_no  4.391707
1               age  7.820284
2               bmi  9.754090
3          children  1.798913
4  region_northeast  1.289267


In [118]:
#Prueba de Durbin-Watson
MODEL=ols('charges~smoker_no+age+bmi+children+region_northeast',data=data).fit()
print(MODEL.summary())
D_W=durbin_watson(MODEL.resid)
print(D_W)
#Entre más cercano a 2 mejor

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     801.0
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:35   Log-Likelihood:                -13549.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1332   BIC:                         2.714e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          1.13e+04   1011.930  

In [119]:
#Prueba de normalidad de los errores
errors=fitted1-data[outcome1]
kstest(errors, 'norm')
#El p-value es menor a 0.05 por lo tanto existe suficiente evidencia de que los errores NO
#siguen una distribución normal

KstestResult(statistic=1.0, pvalue=0.0)

In [120]:
#Prueba de homocedasticidad
white_test = het_white(MODEL.resid,  MODEL.model.exog)
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
print(dict(zip(labels, white_test)))
#el valor importante es el "Test Statistic p-value" aquí si este es menor a 0.05 entonces existe 
#sufciente evidencia estadística de que hay heterocedsticidad

{'Test Statistic': 132.76393197379943, 'Test Statistic p-value': 1.5716267020643833e-19, 'F-Statistic': 8.071983706903623, 'F-Test p-value': 1.1583252650083604e-20}


## Modelo log-level

In [121]:
data['log_age']=np.log(data['age'])
data['log_bmi']=np.log(data['bmi'])
data['log_charges']=np.log(data['charges'])
data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,log_age,log_bmi,log_charges
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1,2.944439,3.328627,9.734176
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0,2.890372,3.519573,7.453302
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0,3.332205,3.496508,8.400538
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0,3.496508,3.122585,9.998092
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0,3.465736,3.363149,8.260197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0,3.912023,3.433019,9.268661
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0,2.890372,3.463233,7.698927
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0,2.890372,3.606856,7.396233
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1,3.044522,3.250374,7.604867


In [122]:
log_predictors=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes','region_northeast','region_northwest','region_southeast','region_southwest']
log_outcome=['log_charges']
log_model=LinearRegression()
log_model.fit(data[log_predictors],data[log_outcome])
print('Intercept',log_model.intercept_)
log_coeficientes=log_model.coef_.tolist()[0]
for name, coef in zip(log_predictors,log_coeficientes):
    print(f'{name}:{coef}')

Intercept [7.68252713]
age:0.03458164053172366
bmi:0.013374822725804434
children:0.10185684855526703
sex_female:0.037708222121644604
sex_male:-0.03770822212164474
smoker_no:-0.7771613929487877
smoker_yes:0.7771613929487879
region_northeast:0.08748413289555422
region_northwest:0.023696569257015163
region_southeast:-0.06971261320991584
region_southwest:-0.04146808894265351


In [123]:
model_log=sm.OLS(data[log_outcome],data[log_predictors].assign(const=1))
results_log=model_log.fit()
results_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_charges   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     549.8
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:36   Log-Likelihood:                -808.52
No. Observations:                1338   AIC:                             1635.
Df Residuals:                    1329   BIC:                             1682.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                  0.0346      0.001     39.655      0.000       0.033       0.036
bmi                  0.0134      0.002      6.381      0.000       0.009       0.017
children             0.1019      0.010     10.085      0.000       0.082       0.122
sex_female           1.7449      0.020     88.435      0.000       1.706       1.784
sex_male             1.6695      0.020     82.844      0.000       1.630       1.709
smoker_no            0.9301      0.021     44.948      0.000       0.889       0.971
smoker_yes           2.4844      0.023    108.119      0.000       2.439       2.529
region_northeast     0.9411      0.022     42.754      0.000       0.898       0.984
region_northwest     0.8773      0.022     39.725      0.000       0.834       0.921
region_southeast     0.7839      0.024     32.338      0.000       0.736       0.831
region_southwest     0.8121      0.023     35.639      0.000       0.767       0.857
const                3.4145      0.032    108.219      0.000       3.353       3.476
==============================================================================
Omnibus:                      463.882   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1673.760
Skew:                           1.679   Prob(JB):                         0.00
Kurtosis:                       7.330   Cond. No.                     5.17e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.32e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [124]:
rows=len(data.index)
log_fitted = log_model.predict(data[log_predictors])
log_RMSE = np.sqrt(mean_squared_error(data[log_outcome],log_fitted))
log_r2 = r2_score(data[log_outcome], log_fitted)
print(f'RMSE: {log_RMSE:.0f}')
print(f'r2: {log_r2:.4f}')
print(rows)
log_adj_r2=1-(1-r2)*((rows-1)/(rows-len(log_predictors)-1))
print(f'adj_r2:{log_adj_r2:.4f}')

RMSE: 0
r2: 0.7679
1338
adj_r2:0.7488


In [125]:
#stepwise selection


y=data[log_outcome]
X=data[log_predictors]

def train_model(variables):
    if len(variables)==0:
        return None
    model=LinearRegression()
    model.fit(X[variables],y)
    return model

def score_model(model,variables):
    if len(variables)==0:
        return AIC_score(y,[y.mean()]*len(y),model,df=1)
    return AIC_score(y,model.predict(X[variables]),model)

best_model,best_variables= stepwise_selection(X.columns, train_model,score_model,verbose=True)
print(best_model.intercept_)
print('Coefficients')
for name, coef in zip(best_variables,best_model.coef_):
    print(name,coef)

Variables: age, bmi, children, sex_female, sex_male, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest
Start: score=3575.57, constant
Step: score=2794.84, add smoker_no
Step: score=1777.52, add age
Step: score=1683.25, add children
Step: score=1656.02, add bmi
Step: score=1640.01, add region_northeast
Step: score=1630.47, add sex_male
Step: score=1623.71, add region_northwest
Step: score=1623.03, add region_southeast
Step: score=1623.03, unchanged None
[8.45592865]
Coefficients
smoker_no [-1.55432279  0.03458164  0.10185685  0.01337482  0.12895222 -0.07541644
  0.06516466 -0.02824452]


In [126]:
log_predictors1=['age','bmi','children','sex_male','smoker_no','region_northeast','region_northwest','region_southeast']
log_outcome1=['log_charges']
log_model2=sm.OLS(data[log_outcome1],data[log_predictors1].assign(const=1))
resultslog=log_model2.fit()
resultslog.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_charges   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     549.8
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:38   Log-Likelihood:                -808.52
No. Observations:                1338   AIC:                             1635.
Df Residuals:                    1329   BIC:                             1682.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                  0.0346      0.001     39.655      0.000       0.033       0.036
bmi                  0.0134      0.002      6.381      0.000       0.009       0.017
children             0.1019      0.010     10.085      0.000       0.082       0.122
sex_male            -0.0754      0.024     -3.091      0.002      -0.123      -0.028
smoker_no           -1.5543      0.030    -51.333      0.000      -1.614      -1.495
region_northeast     0.1290      0.035      3.681      0.000       0.060       0.198
region_northwest     0.0652      0.035      1.863      0.063      -0.003       0.134
region_southeast    -0.0282      0.034     -0.819      0.413      -0.096       0.039
const                8.4559      0.078    107.835      0.000       8.302       8.610
==============================================================================
Omnibus:                      463.882   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1673.760
Skew:                           1.679   Prob(JB):                         0.00
Kurtosis:                       7.330   Cond. No.                         343.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [127]:
log_predictors2=['age','children','sex_male','smoker_no','region_northeast']
log_outcome2=['log_charges']
log_model3=sm.OLS(data[log_outcome2],data[log_predictors2].assign(const=1))
resultslog2=log_model3.fit()
resultslog2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            log_charges   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     845.8
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:39   Log-Likelihood:                -829.70
No. Observations:                1338   AIC:                             1671.
Df Residuals:                    1332   BIC:                             1703.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                  0.0352      0.001     40.082      0.000       0.034       0.037
children             0.1029      0.010     10.044      0.000       0.083       0.123
sex_male            -0.0684      0.025     -2.766      0.006      -0.117      -0.020
smoker_no           -1.5504      0.031    -50.595      0.000      -1.610      -1.490
region_northeast     0.0920      0.029      3.195      0.001       0.035       0.148
const                8.8500      0.047    186.332      0.000       8.757       8.943
==============================================================================
Omnibus:                      447.224   Durbin-Watson:                   2.051
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1536.127
Skew:                           1.634   Prob(JB):                         0.00
Kurtosis:                       7.108   Cond. No.                         175.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Cross-Validation

In [128]:
k=10

In [129]:
slices=np.array_split(data,k)
slices

[     age     bmi  children      charges  sex_female  sex_male  smoker_no  \
 0     19  27.900         0  16884.92400           1         0          0   
 1     18  33.770         1   1725.55230           0         1          1   
 2     28  33.000         3   4449.46200           0         1          1   
 3     33  22.705         0  21984.47061           0         1          1   
 4     32  28.880         0   3866.85520           0         1          1   
 ..   ...     ...       ...          ...         ...       ...        ...   
 129   38  34.700         2   6082.40500           0         1          1   
 130   59  26.505         0  12815.44495           1         0          1   
 131   61  22.040         0  13616.35860           1         0          1   
 132   53  35.900         2  11163.56800           1         0          1   
 133   19  25.555         0   1632.56445           0         1          1   
 
      smoker_yes  region_northeast  region_northwest  region_southeast  \


In [130]:
rsquared_results = []
f_statistic_results = []
p_values_results = []

for i in range(len(slices)):
    reserved_data = slices[i]
    left_side = slices[:i]
    right_side = slices[i+1:]
    remaining_data = pd.concat(left_side+right_side) # Reto: Unir dos partes

    #Entrenamiento
    model = LinearRegression()
    model.fit(remaining_data[log_predictors2],remaining_data[log_outcome2])

    rsquared = model.score(reserved_data[log_predictors2],reserved_data[log_outcome2])
    f_statistic,p_values = f_regression(reserved_data[log_predictors2],reserved_data[log_outcome2])

    rsquared_results.append(rsquared)
    f_statistic_results.append(f_statistic)
    p_values_results.append(p_values)

C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

In [131]:
rsquared_results # Promediar los resultados
mean_r2 = sum(rsquared_results)/len(rsquared_results)
mean_r2

0.7545359877044157

## Pruebas de supuestos de Gauss-Markov

In [132]:
#Prueba de linealidad de Ramsey
log_reg=rg.OLS(data[log_outcome2],data[log_predictors2],hasconst=bool).fit()
log_reset=dg.linear_reset(log_reg,power=2,test_type='fitted',use_f=True)
print('Ramsay_Reset Test P-Value:', np.round(log_reset.pvalue,6))
#El p-value es menor a 0.05 entonces existe suficiente evidencia de que la forma funcional NO
#es correcta, esto se puede deber a una mala especificación del modelo o variables omitidas

Ramsay_Reset Test P-Value: 0.0


C:\Users\hp RED\AIfundamentals\venv\lib\site-packages\statsmodels\stats\diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


In [133]:
log_vif_data=pd.DataFrame()
log_vif_data["feature"]=data[log_predictors2].columns
log_vif_data["VIF"]=[variance_inflation_factor(data[log_predictors2].values,i)
                for i in range(len(data[log_predictors2].columns))]
print(log_vif_data)
#Si el valor es mayor a 10, significa que la varianza de las variables incrementa mucho con el tiempo,
#de esta forma, provocaría multicolinealidad en el modelo
#Aquí quitaste bmi

            feature       VIF
0               age  4.400587
1          children  1.751731
2          sex_male  1.835782
3         smoker_no  3.596571
4  region_northeast  1.289802


In [134]:
#Prueba de Durbin-Watson
LOG_MODEL=ols('log_charges~smoker_no+age+children+region_northeast+sex_male',data=data).fit()
print(MODEL.summary())
D_W=durbin_watson(MODEL.resid)
print(D_W)
#Entre más cercano a 2 mejor

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     801.0
Date:                Thu, 26 May 2022   Prob (F-statistic):               0.00
Time:                        00:37:40   Log-Likelihood:                -13549.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1332   BIC:                         2.714e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          1.13e+04   1011.930  

In [135]:
#Prueba de normalidad de los errores
log_model4=LinearRegression()
log_model4.fit(data[log_predictors2],data[log_outcome2])
log_fitted1 = log_model4.predict(data[log_predictors2])
log_errors=log_fitted1-data[log_outcome2]
kstest(errors, 'norm')
#El p-value es menor a 0.05 por lo tanto existe suficiente evidencia de que los errores NO
#siguen una distribución normal

KstestResult(statistic=1.0, pvalue=0.0)

In [136]:
#Prueba de homocedasticidad
white_test = het_white(LOG_MODEL.resid,  LOG_MODEL.model.exog)
labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
print(dict(zip(labels, white_test)))
#el valor importante es el "Test Statistic p-value" aquí si este es menor a 0.05 entonces existe 
#sufciente evidencia estadística de que hay heterocedsticidad

{'Test Statistic': 132.5278955432647, 'Test Statistic p-value': 6.10347309560429e-20, 'F-Statistic': 8.536407655525108, 'F-Test p-value': 4.3492379112021096e-21}
